In [1]:
import tensorflow as tf
# 获取数据
import tensorflow.examples.tutorials.mnist.input_data as input_data
log_dir = 'log_dir/'

max_steps = 1000
learning_rate = 0.001
dropout = 0.9

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# 定义输入
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784],name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10],name='y-input')
    
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10)


In [3]:
# 初始化
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [4]:
# 初始化
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [5]:
# 定义数据汇总函数
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)       

In [6]:
# 定义多层神经网络
def nn_layer(input_tensor,input_dim,output_dim,layer_name,act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            # 初始化
            weights = weight_variable([input_dim,output_dim])
            # 汇总var信息
            variable_summaries(weights)
        with tf.name_scope('biases'):
            # 初始化
            biases = bias_variable([output_dim])
            # 汇总var信息
            variable_summaries(biases)
        with tf.name_scope('WX_plus_b'):
            preactivate = tf.matmul(input_tensor,weights) + biases
            tf.summary.histogram('pre_activations',preactivate)
        activations = act(preactivate,name='activation')
        tf.summary.histogram('activations',activations)
        return activations

In [7]:
hidden1 = nn_layer(x,784,500,'layer1')

In [8]:
# 创建dropout层
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep-probability',keep_prob)
    # dropout层
    dropped = tf.nn.dropout(hidden1,keep_prob)


In [9]:
# 定义输出层
# 输入是上一层的500个节点，输出是10个分类
# 输出激活函数使用全等映射identit
y = nn_layer(dropped,500,10,'layer2',act=tf.identity)

In [10]:
# 定义损失 使用交叉熵
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=y_)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy',cross_entropy)

<tf.Tensor 'cross_entropy_1:0' shape=() dtype=string>

In [11]:
# 定义优化器
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
tf.summary.scalar('accuracy',accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [12]:
# 汇总summary
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(log_dir + '/train',sess.graph)

In [13]:
test_writer = tf.summary.FileWriter(log_dir + '/test')
tf.global_variables_initializer().run()

In [14]:
# 定义feed_dict
def feed_dict(train):
    if train:
        xs,ys = mnist.train.next_batch(100)
        k = dropout
    else:
        xs,ys = mnist.test.images,mnist.test.labels
        k = 1.0
    return {x:xs,y_:ys,keep_prob:k}

In [15]:
# 创建模型保存器，保存模型
saver = tf.train.Saver()
for i in range(max_steps):
    # 每执行10步，执行数据汇总，写入log文件
    if i % 10 == 0:
        summary,acc = sess.run([merged,accuracy],feed_dict=feed_dict(False))
        # 写入log
        test_writer.add_summary(summary,i)
        print "Accuracy at step %s:%s" % (i,acc)
    else:
        if i% 100 == 99:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary,_ = sess.run([merged,train_step],feed_dict=feed_dict(True),options=run_options,run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata,'step%03d'%i)
            train_writer.add_summary(summary,i)
            saver.save(sess,log_dir + "/model.ckpt",i)
            print "Adding run metadata for ",i
        else:
            summary,_ = sess.run([merged,train_step],feed_dict=feed_dict(True))
            train_writer.add_summary(summary,i)
            
train_writer.close()
test_writer.close()

Accuracy at step 0:0.1212
Accuracy at step 10:0.697
Accuracy at step 20:0.7998
Accuracy at step 30:0.8648
Accuracy at step 40:0.8838
Accuracy at step 50:0.8893
Accuracy at step 60:0.8986
Accuracy at step 70:0.9023
Accuracy at step 80:0.9105
Accuracy at step 90:0.9163
Adding run metadata for  99
Accuracy at step 100:0.9116
Accuracy at step 110:0.9209
Accuracy at step 120:0.9269
Accuracy at step 130:0.9252
Accuracy at step 140:0.9237
Accuracy at step 150:0.9245
Accuracy at step 160:0.925
Accuracy at step 170:0.9216
Accuracy at step 180:0.9309
Accuracy at step 190:0.9334
Adding run metadata for  199
Accuracy at step 200:0.9322
Accuracy at step 210:0.9312
Accuracy at step 220:0.9398
Accuracy at step 230:0.9404
Accuracy at step 240:0.9406
Accuracy at step 250:0.9411
Accuracy at step 260:0.9411
Accuracy at step 270:0.944
Accuracy at step 280:0.9447
Accuracy at step 290:0.9443
Adding run metadata for  299
Accuracy at step 300:0.9428
Accuracy at step 310:0.9481
Accuracy at step 320:0.9435
Accu